In [114]:
import numpy as np

def perturb_with_gaussian_noise_2(coordinates1, coordinates2, std_x, std_y, std_z):
    

    # Unpack the input coordinates
    a, b, c = coordinates1
    x, y, z = coordinates2

    # Generate random perturbations from Gaussian distributions
    perturb_a = np.random.normal(loc=0, scale=std_x)
    perturb_b = np.random.normal(loc=0, scale=std_y)
    perturb_c = np.random.normal(loc=0, scale=std_z)
    perturb_x = np.random.normal(loc=0, scale=std_x)
    perturb_y = np.random.normal(loc=0, scale=std_y)
    perturb_z = np.random.normal(loc=0, scale=std_z)

    # Apply perturbations to the coordinates
    a_perturbed = a + perturb_a
    b_perturbed = b + perturb_b
    c_perturbed = c + perturb_c
    x_perturbed = x + perturb_x
    y_perturbed = y + perturb_y
    z_perturbed = z + perturb_z

    # Return perturbed coordinates as a tuple
    return ([a_perturbed, b_perturbed, c_perturbed], [x_perturbed, y_perturbed, z_perturbed])

In [115]:
# Test data
coordinates1 = (1.0, 2.0, 3.0)
coordinates2 = (4.0, 5.0, 6.0)
std_x = 0.1
std_y = 0.2
std_z = 0.3

# Perform the function call
coordinates1, coordinates2 = perturb_with_gaussian_noise_2(coordinates1, coordinates2, std_x, std_y, std_z)
print(result1, result2)

[0.9279358567968706, 2.254658168484239, 3.2925082651179434] [4.013032795115474, 4.877193668788577, 6.176956792315416]


In [116]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor_coverage_2(nrrd_file, center1, radii1, center2, radii2):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the 1st ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center1)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii1[0] / Spacing[0], radii1[1] / Spacing[1], radii1[2] / Spacing[2])
    
    # Create the 2nd ellipsoid source
    sphere2 = vtk.vtkImageEllipsoidSource()
    sphere2.SetOutputScalarTypeToShort()
    sphere2.SetCenter(center2)
    Spacing = image.GetSpacing()
    sphere2.SetRadius(radii2[0] / Spacing[0], radii2[1] / Spacing[1], radii2[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()
    sphere2.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere2.Update()

    # Align the 1st iceball and tumor images
    sp1 = sphere1.GetOutput()
    sp1.SetOrigin(image.GetOrigin())
    
    # Align the 2nd iceball and tumor images
    sp2 = sphere2.GetOutput()
    sp2.SetOrigin(image.GetOrigin())

    # Logic AND operation between 1st iceball and tumor
    logic1 = vtk.vtkImageLogic()
    logic1.SetInput1Data(image)
    logic1.SetInput2Data(sp1)
    logic1.SetOperationToAnd()
    logic1.SetOutputTrueValue(1)
    logic1.Update()
    
    # Logic AND operation between 2nd iceball and tumor
    logic2 = vtk.vtkImageLogic()
    logic2.SetInput1Data(image)
    logic2.SetInput2Data(sp2)
    logic2.SetOperationToAnd()
    logic2.SetOutputTrueValue(1)
    logic2.Update()

    # Logic OR between (1st iceball and tumor) AND (2nd iceball and tumor)
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(logic1.GetOutput())
    logic.SetInput2Data(logic2.GetOutput())
    logic.SetOperationToOr()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

In [117]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center1 = [123, 120, 22]
radii1 = [4.0, 4.0, 4.0]
center2 = [123, 120, 22]
radii2 = [4.0, 4.0, 4.0]

# Expected output: ratio value (floating-point number)
tumor_coverage_2(nrrd_file, center1, radii1, center2, radii2)

208

In [118]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor(nrrd_file):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()
    image = reader.GetOutput()

    # Logic AND operation between iceball and tumor
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(image)
    logic.SetInput2Data(image)
    logic.SetOperationToAnd()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

In [119]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
tumor(nrrd_file)

1780

In [91]:
def iceball_coverage_2(nrrd_file, center1, radii1, center2, radii2):
    numerator = tumor_coverage_2(nrrd_file, center1, radii1, center2, radii2)
    denominator = tumor(nrrd_file)
    ratio = numerator / denominator
    return ratio
    

In [92]:
nrrd = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
c1 = [123, 120, 22]
r1 = [10.0, 10.0, 10.0]
c2 = [123, 120, 22]
r2 = [10.0, 10.0, 10.0]
iceball_coverage_2(nrrd, c1, r1, c2, r2)

0.8943820224719101

In [120]:
'''
Uses SD to find needle coordinates, finds tumor coverage, if tumor not entirely covered add a counter

Arguments: 
    - nrrd file
    - standard deviation in x, y, z directions
    - iceball center, tuple or array with 3 values
    - iceball radii, tuple or array with 3 values

Return: counter / simulation #
1. Loop through # times
2. Perturbed Coordinates
3. Tumor Ratio
4. If below 1.0
    Add to a counter
else keep on looping

Return percentage of tumor coverage

'''
# iterating Gaussian noise --> need start back at original coordinates
def perturbed_tumor_percentage_2(nrrd, simulate, sd_x, sd_y, sd_z, center1, radii1, center2, radii2):
    counter = 0
    for i in range (simulate):
        center_SD1, center_SD2 = perturb_with_gaussian_noise_2(center1, center2, std_x, std_y, std_z)
        print(center_SD1)
        print(center_SD2)
        ratio = iceball_coverage_2(nrrd, center_SD1, radii1, center_SD2, radii2)
        print(ratio)
        if ratio < 1.0:
            counter += 1
    print (counter)
    print(f"Tumor Coverage Percentage: {counter / simulate * 100:.2f}%")

In [121]:
nrrd = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
simulate = 500
center1 = [111, 121, 22]
radii1 = [4.0, 4.0, 4.0]
center2 = [126, 117, 22]
radii2 = [4.0, 4.0, 4.0]
perturbed_tumor_percentage_2(nrrd, simulate, 2.0, 2.0, 2.0, center1, radii1, center2, radii2)

[111.08643714334185, 120.878583707339, 21.876821689262513]
[126.10809089687477, 116.82484728786704, 21.85748340459677]
0.25280898876404495
[110.84813730925647, 121.0780377093586, 21.796701279018833]
[125.87463687310145, 116.86642594069504, 21.98440979801543]
0.2539325842696629
[111.14374215372523, 121.24228448758461, 22.203677231650968]
[126.11163021605684, 117.33490048342216, 22.00022358002498]
0.2550561797752809
[111.0857156828522, 120.94354812664626, 21.677941035164675]
[125.99661205028582, 117.02526092822883, 22.241227709837947]
0.2393258426966292
[110.96482400814652, 121.18200149224569, 21.913196076111486]
[125.95541374363022, 117.20673754658134, 22.20850906930092]
0.2550561797752809
[110.97287305939474, 121.01109638425797, 22.261400591515415]
[125.95091679886413, 116.75449504308001, 21.753013650421803]
0.23820224719101124
[111.05355071229025, 120.67802157558604, 21.97499136450528]
[125.93483419878292, 116.91021590434804, 22.175845002789558]
0.25280898876404495
[111.18547604398304

0.24382022471910111
[111.07433607108084, 121.34114591286671, 22.18449271304489]
[125.9470748513537, 117.2252181064331, 22.073446501052196]
0.2533707865168539
[111.01730965795649, 120.86780598343758, 22.228875238105253]
[125.98688144978063, 116.83966046820176, 22.32974011062943]
0.2353932584269663
[110.9517062232935, 120.87508651608832, 21.893708336537095]
[125.97297601152465, 117.03393780562408, 21.90419682454278]
0.25674157303370787
[110.84466967912687, 120.92830535086169, 22.181827453144653]
[126.03208829344496, 117.04186302519498, 22.468420583322455]
0.23370786516853934
[110.9726993610132, 120.6653302282243, 21.857446632278343]
[125.9544437286427, 116.83182100854505, 22.085785086513972]
0.25449438202247193
[111.05573527246638, 121.02759397111079, 21.847026727274205]
[126.10280527907993, 116.99306291667114, 22.131861797713494]
0.25449438202247193
[111.1407336124214, 120.9026452956185, 21.926511026067402]
[125.91961989452669, 117.1094937609428, 21.548248910345993]
0.24213483146067416


0.2410112359550562
[110.99492258377119, 120.90115002321622, 21.734442215972763]
[125.937100000776, 117.09349092834586, 22.244207727465575]
0.2443820224719101
[110.94571652031509, 121.07818562751379, 21.991406693637657]
[125.98858245005299, 117.07109220460795, 21.850908587194507]
0.25674157303370787
[110.88182398167484, 120.9916579175616, 21.764879901771597]
[125.79075582331112, 117.09163119833464, 22.175361055540687]
0.250561797752809
[110.96908198417555, 121.17701028323988, 21.80747703872176]
[125.86327025105615, 117.04835442573379, 21.63992453795057]
0.23876404494382023
[110.97541676966011, 121.13527718078039, 21.96107031257188]
[126.03207466984237, 116.78576295459428, 21.64384739625126]
0.2455056179775281
[111.0842113619399, 121.15637045073584, 22.280827089708797]
[125.93310566373134, 117.14517903920155, 21.931463976595484]
0.24662921348314606
[111.0895188694422, 121.19821478938577, 21.667583788943087]
[126.0407647878099, 116.4688506347551, 22.253034009505022]
0.2410112359550562
[11

0.21853932584269664
[110.89199301697904, 120.9623578727691, 21.538067118720402]
[125.8159457662781, 116.95151660465744, 22.437785307044845]
0.2303370786516854
[110.92342198797775, 121.0616785396724, 21.691407965503426]
[126.03826482329116, 117.17522099305226, 22.40129278757558]
0.23426966292134832
[110.93845907665826, 120.99411867086414, 21.87532390047026]
[125.99378156025416, 117.01009810460214, 21.641848517649017]
0.2393258426966292
[110.87043428265396, 120.9200453432389, 22.180202728540866]
[125.95430834162974, 116.90093794170959, 22.323254670294713]
0.23426966292134832
[110.83262134933933, 120.99250510944532, 21.657918650423834]
[126.10512201942677, 117.04445434559656, 22.343936486082978]
0.2303370786516854
[110.97976580763005, 121.11862716969765, 21.73709563442489]
[126.05812353085553, 116.96077954068369, 21.96165979729327]
0.250561797752809
[110.9137377297755, 120.88400025697456, 22.336024812768738]
[126.06584920606699, 117.2553858561466, 22.137651386660984]
0.23820224719101124
[

0.251685393258427
[110.91205518698219, 120.92650806478159, 21.817557499785956]
[126.04891225012048, 116.87134062670039, 21.439440822140682]
0.23426966292134832
[110.78127480720248, 120.90128080699975, 21.649334395229282]
[126.02171038916894, 117.1234577319115, 21.68780107676202]
0.22865168539325842
[111.05545606485471, 121.01881505365148, 21.554308022060216]
[126.13563575947073, 117.04261083941822, 21.876445802517914]
0.24269662921348314
[111.0677495303217, 121.08733039005104, 21.70922922015679]
[126.01182189235712, 116.90447121918177, 22.08540761572729]
0.251685393258427
[111.01202173429397, 121.05752778622794, 21.902578843085045]
[125.79909047251344, 116.97673583776887, 22.15550179516602]
0.25786516853932584
[111.11877487393339, 120.59818244710476, 22.566882461481445]
[125.83232275136267, 117.20956129386138, 22.11423022955117]
0.23426966292134832
[110.96987009747946, 120.79949245070462, 22.011691513177546]
[125.9244700136783, 116.56372774296851, 21.865266968802466]
0.2539325842696629

0.2443820224719101
[111.1242056782273, 121.11904087294207, 22.01736304816001]
[125.93420140037647, 117.49179401474419, 22.098675870253345]
0.2612359550561798
[110.97624694266251, 120.95347551956492, 22.039214114754923]
[125.98373674068235, 117.07359279184176, 22.095016545493323]
0.25617977528089886
[111.0741114918335, 120.6862085739924, 22.478938200033205]
[125.99404560461404, 117.15855209792349, 22.679803741521088]
0.22134831460674156
[110.95681153874231, 120.77627608671217, 22.01848757180916]
[125.98744282158918, 116.78175835288094, 21.92361246104905]
0.2556179775280899
[111.30756547197473, 121.29975544346695, 22.47031534202785]
[125.91131045879075, 116.71530136697893, 21.7958655341853]
0.2410112359550562
[111.12665673362805, 120.93988336480427, 21.92477865467847]
[125.94130717666432, 117.10853867225764, 21.774505953099982]
0.2556179775280899
[110.82759469590026, 120.80100041457942, 22.186051058498833]
[125.9872615281572, 117.46871491618259, 21.97567378759456]
0.24831460674157305
[11

0.24887640449438203
[110.9485594392261, 120.82864014180028, 22.1508608357641]
[125.98355731613378, 116.9268082510331, 21.779270202390695]
0.24606741573033708
[111.01632805048354, 121.01217041712769, 22.39102746262712]
[125.82763930903657, 116.6721659213875, 22.0791394275347]
0.23876404494382023
[111.03345956586158, 120.6448749547769, 22.221642455455484]
[125.81363961275028, 116.94021841092425, 22.089762915901765]
0.24606741573033708
[111.01104889449348, 121.01526799821418, 21.714450614315762]
[126.07484820240622, 116.93204561048003, 22.314728684406617]
0.23820224719101124
[111.22416712270953, 120.99619870622932, 22.324183273086913]
[125.96939989427814, 117.41922801019454, 21.739539585447904]
0.23202247191011235
[110.97180242861937, 120.8425534725736, 22.417317051849412]
[125.93153698363851, 116.84927213049723, 22.228658421214337]
0.23370786516853934
[111.11310595366623, 121.26495099488383, 22.264309766603134]
[126.2138933320141, 116.99568348832076, 21.637051615559958]
0.232584269662921

0.24382022471910111
[111.14025369810341, 121.16071086685326, 22.056078516642366]
[125.96634537338673, 116.83703656485525, 22.020375835598365]
0.2606741573033708
[110.98901288989696, 120.850764530304, 22.175117402206148]
[126.27966004650534, 116.68381580821608, 21.990176975705793]
0.24831460674157305
[110.98935184482923, 120.88722552469405, 22.47193564271015]
[125.9102010319603, 117.02575202874914, 22.234661746628028]
0.23314606741573032
[110.8449908252226, 121.03622061624871, 22.207713204008368]
[126.02643697319637, 117.30336412259119, 21.807813892091364]
0.24157303370786518
[111.06782521929097, 120.85903063075995, 21.622981559727755]
[125.97692656209615, 117.08527784434547, 21.68391621105666]
0.22865168539325842
[110.92593410783233, 120.7117496240479, 21.895440311814994]
[126.01991657235304, 117.11347409495012, 21.38362689714459]
0.23595505617977527
[111.05129932091376, 121.0839251961198, 22.27373947305754]
[125.92077347478799, 117.11534265954873, 22.239060692223966]
0.242134831460674

0.24887640449438203
[111.02314435638127, 120.89578526307804, 21.908639441867997]
[125.89966772374085, 117.078933777822, 21.665936367156842]
0.24213483146067416
[110.96034023365851, 121.01857803649725, 22.628170413533887]
[125.96658205595307, 116.78929303890467, 21.447402388649042]
0.2196629213483146
[110.84375243226353, 120.77550976400413, 22.035794104913577]
[126.07663109730446, 117.23616389358548, 22.386224170459677]
0.24325842696629213
[110.94530414553947, 121.0891129849115, 22.05594688604404]
[125.8363361107598, 117.13246272512366, 22.130180810026467]
0.2550561797752809
[110.90467255349324, 120.721752431954, 22.47428292320157]
[126.06161134023306, 117.12617996819291, 21.95048974531898]
0.23651685393258426
[110.96485731138281, 120.76459717627303, 22.220513581303088]
[125.89954785704049, 116.80346404076036, 21.397576099247622]
0.22977528089887642
[110.99796154639519, 120.87405934095733, 21.915132400654887]
[125.98194104472691, 116.71996873585896, 22.482580633312082]
0.243820224719101